In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.cluster import AgglomerativeClustering
from sklearn.preprocessing import StandardScaler
from scipy.cluster.hierarchy import dendrogram, linkage

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# Load the dataset with the specified encoding
retail = pd.read_csv("/kaggle/input/online-retail-customer-clustering/OnlineRetail.csv", encoding="ISO-8859-1")

# Display the first few rows of the dataset
retail.head()


In [ ]:
#dropping rows with 0 customer id
retail = retail.dropna(subset=['CustomerID'])

In [ ]:
retail.describe()

In [ ]:
#as minimum of quantity is neagative, we remove rows with <=0 quantity. 
# Display the original number of rows
print("Original number of rows:", retail.shape[0])
# Remove entries where Quantity is 0 or less
retail = retail[retail['Quantity'] > 0]
# Display the number of rows after filtering
print("Number of rows after filtering:", retail.shape[0])

# Verify that all remaining Quantity values are positive
print("All quantities are positive:", (retail['Quantity'] > 0).all())

In [ ]:
retail.describe()

In [ ]:
retail.head(25)

We see that several invoices per each customer. each invoice itslef has many items in the order, so we will multiply each quantity with unit price.

In [ ]:

# Creating a new column by multiplying 'Quantity' and 'UnitPrice'
retail['Paid'] = retail['Quantity'] * retail['UnitPrice']

# Displaying the updated DataFrame
retail.head()


In [ ]:
#we see same invoice has many items, so we will add the paid amount of each invoice
rfm_details = retail.groupby(['CustomerID','InvoiceNo']).agg({'Paid': 'sum'}).reset_index()
rfm_details.head()

In [ ]:
#how much each customer paid (sum of all invoices)
rfm_details2 = rfm_details.groupby(['CustomerID']).agg({'Paid':'sum'}).reset_index()
rfm_details2.head()

In [ ]:
#how many times each customer ordered (no. of unique invoices or frequency)
frequency_data = retail.groupby('CustomerID')['InvoiceNo'].nunique().reset_index()
frequency_data.columns = ['CustomerID', 'Frequency']
rfm_details2 = pd.merge(rfm_details2, frequency_data, on='CustomerID', how='left')
rfm_details2.head()

In [ ]:
#making a copy of df
rfm_details3 = rfm_details2.copy()
rfm_details3.head()

In [ ]:
#lets find most recent transaction and then find difference of number of days from that to the last transaction for each customer for **recency**
last_date = max(retail['InvoiceDate'])
print(last_date)

In [ ]:
retail_copy = retail.copy()
retail_copy.head()

In [ ]:
#converting datetime to standard format
retail_copy['InvoiceDate'] = pd.to_datetime(retail_copy['InvoiceDate'], format='%d-%m-%Y %H:%M')
retail_copy.head()

In [ ]:
max_date = retail_copy['InvoiceDate'].max()

# Calculate the difference between the maximum date and each 'InvoiceDate'
retail_copy['diff'] = max_date - retail_copy['InvoiceDate']

retail_copy.head()

In [ ]:
retail_copy['diff_days'] = retail_copy['diff'].dt.days

# Find the minimum 'diff_days' for each 'CustomerID'
min_diff_per_customer = retail_copy.groupby('CustomerID')['diff_days'].min().reset_index()
min_diff_per_customer.columns = ['CustomerID', 'time']
retail_copy['diff_days'] = retail_copy['diff'].dt.days

# Find the minimum 'diff_days' for each 'CustomerID'
min_diff_per_customer = retail_copy.groupby('CustomerID')['diff_days'].min().reset_index()
min_diff_per_customer.columns = ['CustomerID', 'time']

retail_copy.head()

In [ ]:
rfm_details3 = pd.merge(rfm_details3, min_diff_per_customer, on='CustomerID', how='left')
rfm_details3.head()

In [ ]:
columns_to_scale = ['Paid', 'Frequency', 'time']

# Initialize the MinMaxScaler
scaler = MinMaxScaler()

# Scale the specified columns
rfm_details3[columns_to_scale] = scaler.fit_transform(rfm_details3[columns_to_scale])

# Create the new DataFrame rfm_details4 with the scaled values and include the 'CustomerID' column
rfm_details4 = rfm_details3.copy()

rfm_details4.head()

In [ ]:

# Suppress warnings
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    
    # Create the pairplot
    sns.pairplot(rfm_details4)
    plt.suptitle('Pairplot of rfm_details4', y=1.02)
    plt.show()


In [ ]:

# Create boxplot for 'Frequency'
plt.figure(figsize=(8, 6))
sns.boxplot(y=rfm_details4['Frequency'], whis=[5, 95])
plt.title('Boxplot of Frequency (whis=[5, 95])')
plt.ylabel('Scaled Values')
plt.show()

# Create boxplot for 'Paid' with smaller scale
plt.figure(figsize=(8, 6))
sns.boxplot(y=rfm_details4['Paid'], whis=[1, 99])
plt.title('Boxplot of Paid (whis=[1, 99])')
plt.ylabel('Scaled Values')
plt.yticks(ticks=plt.yticks()[0], labels=[f"{tick:.2f}" for tick in plt.yticks()[0]])  # Change y-axis ticks to display with 2 decimal places
plt.show()

# Create boxplot for 'time' with different scale
plt.figure(figsize=(8, 6))
sns.boxplot(y=rfm_details4['time'], whis=[10, 90])
plt.title('Boxplot of Time (whis=[10, 90])')
plt.ylabel('Scaled Values')
plt.show()


In [ ]:
correlation_matrix = rfm_details4.corr()

# Create a heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f", linewidths=0.5)
plt.title('Correlation Heatmap')
plt.show()

In [ ]:
# Assuming rfm_details4 DataFrame is already defined
# For demonstration, let's assume the DataFrame is already loaded and has the required columns

# Define columns for outlier detection (excluding 'CustomerID')
scatter_columns = ['Frequency', 'Paid', 'time']

# Calculate quartiles and IQR for specified columns in rfm_details4
Q1 = rfm_details4[scatter_columns].quantile(0.25)
Q3 = rfm_details4[scatter_columns].quantile(0.75)
IQR = Q3 - Q1

# Define lower and upper bounds for outlier detection
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Suppress warnings
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    
    # Filter the data to remove outliers, excluding 'CustomerID'
    filtered_data = rfm_details4.copy()
    for col in scatter_columns:
        filtered_data = filtered_data[(filtered_data[col] >= lower_bound[col]) & (filtered_data[col] <= upper_bound[col])]

    # Create scatter plots for each pair of columns with outliers removed
    sns.pairplot(filtered_data[scatter_columns])
    plt.suptitle('Pairplot with Outliers Removed', y=1.02)
    plt.show()


In [ ]:
filtered_data.head(5)

In [ ]:
# Create boxplot for 'Frequency'
plt.figure(figsize=(8, 6))
sns.boxplot(y=filtered_data['Frequency'], whis=[5, 95])
plt.title('Boxplot of Frequency (whis=[5, 95])')
plt.ylabel('Scaled Values')
plt.show()

# Create boxplot for 'Paid' with smaller scale
plt.figure(figsize=(8, 6))
sns.boxplot(y=filtered_data['Paid'], whis=[1, 99])
plt.title('Boxplot of Paid (whis=[1, 99])')
plt.ylabel('Scaled Values')
plt.yticks(ticks=plt.yticks()[0], labels=[f"{tick:.2f}" for tick in plt.yticks()[0]])  # Change y-axis ticks to display with 2 decimal places
plt.show()

# Create boxplot for 'time' with different scale
plt.figure(figsize=(8, 6))
sns.boxplot(y=filtered_data['time'], whis=[10, 90])
plt.title('Boxplot of Time (whis=[10, 90])')
plt.ylabel('Scaled Values')
plt.show()

In [ ]:
correlation_matrix = filtered_data.corr()

# Create a heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f", linewidths=0.5)
plt.title('Correlation Heatmap')
plt.show()

In [ ]:
filtered_data.head()

In [ ]:
filtered_data2 = filtered_data.copy()

In [ ]:
filtered_data_untouched = filtered_data.copy()

In [ ]:
filtered_data4 = filtered_data_untouched.copy()

In [ ]:
#we have removed outliers, so now lets proceed with clustering.

columns_for_elbow = ['Frequency', 'time', 'Paid']

# Convert DataFrame to array
X = filtered_data[columns_for_elbow].values

# Calculate within-cluster sum of squares (WCSS) for different values of k
wcss = []
for i in range(1, 11):
    kmeans = KMeans(n_clusters=i, init='k-means++', max_iter=300, n_init=10, random_state=0)
    kmeans.fit(X)
    wcss.append(kmeans.inertia_)

# Plot the elbow curve
plt.figure(figsize=(8, 6))
plt.plot(range(1, 11), wcss, marker='o', linestyle='--')
plt.title('Elbow Method for Optimal k')
plt.xlabel('Number of Clusters (k)')
plt.ylabel('Within-Cluster Sum of Squares (WCSS)')
plt.xticks(np.arange(1, 11, 1))
plt.grid(True)
plt.show()

In [ ]:
# Define columns for clustering
cluster_columns = ['Frequency', 'time', 'Paid']

# Convert DataFrame to array
X = filtered_data[cluster_columns].values

# Initialize a list to store silhouette scores
silhouette_scores = []

# Set the range of k values for silhouette analysis
k_range = range(2, 6)

# Suppress warnings
with warnings.catch_warnings():
    warnings.simplefilter("ignore")

    # Perform silhouette analysis for each value of k
    for k in k_range:
        kmeans = KMeans(n_clusters=k, random_state=42)
        cluster_labels = kmeans.fit_predict(X)
        silhouette_avg = silhouette_score(X, cluster_labels)
        silhouette_scores.append(silhouette_avg)

# Print silhouette scores for each value of k
for k, silhouette_avg in zip(k_range, silhouette_scores):
    print(f"For n_clusters = {k}, the average silhouette score is {silhouette_avg:.4f}")


In [ ]:
cluster_columns = ['Frequency', 'time', 'Paid']

# Convert DataFrame to array
X = filtered_data[cluster_columns].values

# Initialize a list to store silhouette scores
silhouette_scores = []

# Set the range of n_clusters values for silhouette analysis
n_clusters_range = range(2, 6)

# Perform silhouette analysis for each value of n_clusters
for n_clusters in n_clusters_range:
    # Perform hierarchical clustering
    hierarchical = AgglomerativeClustering(n_clusters=n_clusters)
    cluster_labels = hierarchical.fit_predict(X)
    silhouette_avg = silhouette_score(X, cluster_labels)
    silhouette_scores.append(silhouette_avg)

# Print silhouette scores for each value of n_clusters
for n_clusters, silhouette_avg in zip(n_clusters_range, silhouette_scores):
    print(f"For n_clusters = {n_clusters}, the average silhouette score is {silhouette_avg:.4f}")

In silhouette test of both kmeans and hierarchical clustering, and elbow test we see 2 clusters will be best.

In [ ]:
# Define the columns for clustering
cluster_columns = ['Frequency', 'time', 'Paid']

# Convert DataFrame to array
X = filtered_data[cluster_columns].values

# Suppress warnings
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    
    # Initialize K-means with 2 clusters
    kmeans = KMeans(n_clusters=2, random_state=42)
    
    # Fit K-means clustering on the data
    kmeans.fit(X)
    
    # Get the cluster labels for each data point
    cluster_labels = kmeans.labels_

# Add the cluster labels to the filtered_data DataFrame
filtered_data['ClusterKMeans'] = cluster_labels

# Print the counts of each cluster
print(filtered_data['ClusterKMeans'].value_counts())

# Display the updated DataFrame with cluster labels
print(filtered_data.head())


In [ ]:

# Suppress warnings
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    
    # Create pairplot with hue set to cluster assignments
    sns.pairplot(filtered_data, hue='ClusterKMeans', palette='viridis')
    plt.suptitle('K-means Clustering Pair Plot', y=1.02)
    plt.show()


In [ ]:
# we will take 2 clusters.

In [ ]:
filtered_data.head()

In [ ]:
filtered_data2.head()

In [ ]:
# Load the data (assuming filtered_data2 is already available)
# filtered_data2 = pd.read_csv('path_to_filtered_data2.csv')

# Define the columns for clustering (excluding 'CustomerID')
cluster_columns = ['Paid', 'Frequency', 'time']

# Standardize the data (optional but recommended for clustering)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(filtered_data2[cluster_columns])

# Perform hierarchical clustering
hierarchical = AgglomerativeClustering(n_clusters=2)
cluster_labels = hierarchical.fit_predict(X_scaled)

# Add cluster labels to the DataFrame under a new column 'cluster_Hier'
filtered_data2['cluster_Hier'] = cluster_labels

# Visualize the dendrogram
linked = linkage(X_scaled, method='ward')
plt.figure(figsize=(10, 7))
dendrogram(linked, labels=filtered_data2['CustomerID'].values, leaf_rotation=90, leaf_font_size=10)
plt.title('Hierarchical Clustering Dendrogram')
plt.xlabel('CustomerID')
plt.ylabel('Distance')
plt.show()

# Create scatter plots to visualize the clusters
plt.figure(figsize=(8, 6))
sns.scatterplot(data=filtered_data2, x='Frequency', y='Paid', hue='cluster_Hier', palette='viridis')
plt.title('Hierarchical Clustering: Frequency vs Paid')
plt.xlabel('Frequency')
plt.ylabel('Paid')
plt.legend(title='Cluster')
plt.show()

plt.figure(figsize=(8, 6))
sns.scatterplot(data=filtered_data2, x='time', y='Paid', hue='cluster_Hier', palette='viridis')
plt.title('Hierarchical Clustering: Time vs Paid')
plt.xlabel('Time')
plt.ylabel('Paid')
plt.legend(title='Cluster')
plt.show()

plt.figure(figsize=(8, 6))
sns.scatterplot(data=filtered_data2, x='time', y='Frequency', hue='cluster_Hier', palette='viridis')
plt.title('Hierarchical Clustering: Time vs Frequency')
plt.xlabel('Time')
plt.ylabel('Frequency')
plt.legend(title='Cluster')
plt.show()

# Display the updated DataFrame
print(filtered_data2.head())


In [ ]:
filtered_data.head()

In [ ]:
filtered_data2.head()

In [ ]:
rfm_details5 = rfm_details2.copy()
rfm_details5= pd.merge(rfm_details2, min_diff_per_customer, on='CustomerID', how='left')

In [ ]:
rfm_details5.head()

In [ ]:
merged1 = pd.merge(filtered_data[['CustomerID', 'ClusterKMeans']], filtered_data2[['CustomerID', 'cluster_Hier']], on='CustomerID', how='inner')
merged2 = pd.merge(merged1, rfm_details5, on='CustomerID', how='inner')

# Select the desired columns
rfm_clustered = merged2[['CustomerID', 'Paid', 'Frequency', 'time', 'ClusterKMeans', 'cluster_Hier']]

# Order by CustomerID
rfm_clustered = rfm_clustered.sort_values(by='CustomerID')

# Reset index (optional)
rfm_clustered = rfm_clustered.reset_index(drop=True)

rfm_clustered.head()

In [ ]:
rfm_clustered.describe()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import warnings

# Suppress warnings
with warnings.catch_warnings():
    warnings.simplefilter("ignore")

    # Create pairplot with hue set to ClusterKMeans
    sns.pairplot(rfm_clustered, hue='ClusterKMeans')
    plt.suptitle('Pairplot with ClusterKMeans Hue', y=1.02)
    plt.show()

    # Create pairplot with hue set to cluster_Hier
    sns.pairplot(rfm_clustered, hue='cluster_Hier')
    plt.suptitle('Pairplot with cluster_Hier Hue', y=1.02)
    plt.show()


In [ ]:

# Suppress warnings
with warnings.catch_warnings():
    warnings.simplefilter("ignore")

    # Combine both cluster labels into a single string column for a single hue
    rfm_clustered['combined_cluster'] = rfm_clustered['ClusterKMeans'].astype(str) + "_" + rfm_clustered['cluster_Hier'].astype(str)

    # Create a pairplot with the combined clusters as hue
    sns.pairplot(rfm_clustered, hue='combined_cluster')
    plt.suptitle('Pairplot with Combined Clusters Hue', y=1.02)
    plt.show()


In [ ]:
rfm_clustered.head()

In [ ]:
rfm_clustered.describe()

In [ ]:
cluster_kmeans_0 = rfm_clustered[rfm_clustered['ClusterKMeans'] == 0]

# Filter rows where ClusterKMeans is 1
cluster_kmeans_1 = rfm_clustered[rfm_clustered['ClusterKMeans'] == 1]

In [ ]:
cluster_kmeans_0.describe()

In [ ]:
cluster_kmeans_1.describe()

In [ ]:
cluster_Hier_0 = rfm_clustered[rfm_clustered['cluster_Hier'] == 0]

# Filter rows where ClusterKMeans is 1
cluster_Hier_1 = rfm_clustered[rfm_clustered['cluster_Hier'] == 1]

In [ ]:
cluster_Hier_0.describe()

In [ ]:
cluster_Hier_1.describe()

**K-Means Clustering**

- Cluster 0:
  This cluster consists of 2736 customers with an average payment of 1030, frequency 3.2, and time of last purchase 41.8 days.
- Cluster 1:
  This cluster consists of 975 customers, with an average payment of 484, frquency of 1.6 and time of last purchase 221 days.
  
*Cluster zero customers are of a higher value to the company as they are more recent, high spender and frequent purchasers.*

**Hierarchical Clustering**

- Cluster 0:
  This cluster consists of 2974 customers with an average payment of 584, frequency 2, and time of last purchase 102 days.
- Cluster 1:
  This cluster consists of 737 customers, with an average payment of 2108, frquency of 6.3 and time of last purchase 33.6 days.
  
*Cluster one customers are of a higher value to the company as they are more recent, high spender and frequent purchasers.*

**Overall Summary**

Kmeans clustering gives us a more general, bird eye view on valuable customers, whereas Hierarchical Clustering is more sharp and pointed outlook on customer of high importance.

# This document ends here, Thanks for reading!